In [25]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [26]:
#data1=pd.read_csv('bs140513_032310.csv')
data1=pd.read_csv('D:\\DataScienceCompetitions\\AI4Hackathon\\synthetic-data-from-a-financial-payment-system\\bs140513_032310.csv')

In [27]:
data1.head()

,step,customer,age,gender,zipcodeOri,merchant,zipMerchant,category,amount,fraud
0,0,'C1093826151','4','M','28007','M348934600','28007','es_transportation',4.55,0
1,0,'C352968107','2','M','28007','M348934600','28007','es_transportation',39.68,0
2,0,'C2054744914','4','F','28007','M1823072687','28007','es_transportation',26.89,0
3,0,'C1760612790','3','M','28007','M348934600','28007','es_transportation',17.25,0
4,0,'C757503768','5','M','28007','M348934600','28007','es_transportation',35.72,0


In [28]:
#zipcodeOri and zipMerchant doesn't have any predictive power, zero variance(unique value)
data1=data1.drop(columns=['zipcodeOri', 'zipMerchant'])

In [29]:
data1.shape

(594643, 8)

In [30]:
df=data1.groupby('customer').agg({'fraud':'sum','category':'count' }).reset_index().\
rename(columns = {'category':'num_transactions', 'fraud':'num_frauds'})

In [31]:
df['ratio_fraud']=df['num_frauds']/df['num_transactions']

In [32]:
data1=pd.merge(data1, df[['customer', 'ratio_fraud']], on='customer')

In [33]:
data1.head()

,step,customer,age,gender,merchant,category,amount,fraud,ratio_fraud
0,0,'C1093826151','4','M','M348934600','es_transportation',4.55,0,0.0
1,1,'C1093826151','4','M','M855959430','es_hyper',14.38,0,0.0
2,2,'C1093826151','4','M','M348934600','es_transportation',37.21,0,0.0
3,3,'C1093826151','4','M','M348934600','es_transportation',31.63,0,0.0
4,4,'C1093826151','4','M','M348934600','es_transportation',35.86,0,0.0


In [34]:
#dropping customer id
data1=data1.drop(columns=['customer'])

In [35]:
x=list(data1.columns)

In [36]:
x.remove('fraud')

In [37]:
x

['step', 'age', 'gender', 'merchant', 'category', 'amount', 'ratio_fraud']

In [38]:
X=data1.loc[:,x].values
y=data1.iloc[:, 6].values

In [39]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [40]:
labelencoder_X1=LabelEncoder()
X[:,1]=labelencoder_X1.fit_transform(X[:,1])

labelencoder_X2=LabelEncoder()
X[:,2]=labelencoder_X2.fit_transform(X[:,2])

labelencoder_X3=LabelEncoder()
X[:,3]=labelencoder_X3.fit_transform(X[:,3])

labelencoder_X4=LabelEncoder()
X[:,4]=labelencoder_X4.fit_transform(X[:,4])


In [41]:
# Encoding categorical data
onehotencoder = OneHotEncoder(categorical_features=[1,2,3,4])
X = onehotencoder.fit_transform(X).toarray()

In [42]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = sc.fit_transform(X)

In [43]:
from sklearn import model_selection
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import VotingClassifier

In [44]:
# create the sub models
estimators = []
classifier_RF=RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=0)
estimators.append(('randomforest', classifier_RF))
classifier_xg = XGBClassifier()
estimators.append(('xgb', classifier_xg))
classifier_NB=GaussianNB()
estimators.append(('NaiveBayes', classifier_NB))


In [45]:
# create the ensemble model
ensemble = VotingClassifier(estimators)


In [47]:
results = model_selection.cross_val_score(ensemble, X, y, cv=10)

C:\Users\gautam\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\gautam\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\gautam\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\gautam\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\labe

In [48]:
print(results.mean())

0.9961254018903409


# Calculate Performance Metrics of the Voting Ensemble 

In [50]:
#Random Forest
from sklearn.ensemble import RandomForestClassifier
classifier_RF=RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=0)
classifier_RF.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [51]:
y_pred_rf=classifier_RF.predict(X_test)

In [52]:
#XGBoost
from xgboost import XGBClassifier
classifier_xg = XGBClassifier()
classifier_xg.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [53]:
y_pred_xg=classifier_xg.predict(X_test)

C:\Users\gautam\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [54]:
#Naive Bayes
from sklearn.naive_bayes import GaussianNB
classifier_NB=GaussianNB()
classifier_NB.fit(X_train, y_train)

GaussianNB(priors=None)

In [55]:
y_pred_nb=classifier_NB.predict(X_test)

In [56]:
y_pred_temp=y_pred_nb + y_pred_xg + y_pred_rf

In [57]:
y_pred=[]
for x in y_pred_temp:
    if x<=1:
        y_pred.append(0)
    else:
        y_pred.append(1)

In [58]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test, y_pred)
cm

array([[117303,    186],
       [   222,   1218]], dtype=int64)

In [59]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, y_pred)

0.9221251032011507

In [60]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

          0       1.00      1.00      1.00    117489
          1       0.87      0.85      0.86      1440

avg / total       1.00      1.00      1.00    118929

